In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#df = pd.read_csv(‘xx.csv’)

In [3]:
path = os.path.join(os.getcwd(), '../raw_data')

data_prep = pd.read_csv(os.path.join(path, 'x_train_preprocessed.csv'))
data_prep

,Unnamed: 0,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,0,good job considering retail,apple good company period pay average working ...,job movement lateral extremely difficult get p...,NaN,good job considering retail apple good company...,31,118,127,0,...,0.600000,0.700000,0.666667,0.516667,1.000000,-0.500000,0.000000,0.000000,0.720000,0.350000
1,1,warning potential new hire,good pay smart coworkers good insurance,hr protect company employee careful restricted...,many employment contract state restricted stoc...,warning potential new hire good pay smart cowo...,31,41,132,1509,...,0.727273,0.068182,0.614286,0.538095,0.625000,0.075000,0.499852,0.120897,0.536005,0.157255
2,2,demanding job get work brilliant mind industry,intelligent co worker encouraged innovate good...,promote work life balance word employee get re...,employee get burned soon life outside work goi...,demanding job get work brilliant mind industry...,87,89,164,150,...,1.000000,0.900000,0.833333,0.783333,0.000000,0.000000,0.368750,-0.156250,0.706250,0.489583
3,3,deliver get fired,pay competitive look good resume move around i...,work life balance hostile environment reward b...,people disposable,deliver get fired pay competitive look good re...,21,189,306,26,...,0.000000,0.000000,0.460714,0.403571,0.492063,0.031746,0.000000,0.000000,0.474150,0.244218
4,4,good salary perk benefit,good salary perk benefit,hectic schedule u ill learning cutting technology,NaN,good salary perk benefit good salary perk bene...,34,34,57,0,...,0.600000,0.700000,0.600000,0.700000,0.950000,-0.550000,0.000000,0.000000,0.775000,0.075000
5,5,happy year working amazon,great people challenging project fast paced en...,hard get often turn away good people left clea...,great hiring need get better,happy year working amazon great people challen...,45,59,108,50,...,1.000000,0.800000,0.783333,0.500000,0.381250,0.127083,0.625000,0.650000,0.612500,0.410833
6,6,one hopefully soon remembered,great experience advancement understanding uti...,always enough time meet deadline,keep positive attitude try cooperate much poss...,one hopefully soon remembered great experience...,35,121,40,80,...,0.000000,0.000000,0.575000,0.450000,0.500000,0.000000,0.772727,0.113636,0.639091,0.225455
7,7,hardwork peanut,fellow team member great work,limited opportunity move within company,try treat team member like robot like human,hardwork peanut fellow team member great work ...,20,44,50,67,...,0.000000,0.000000,0.750000,0.800000,0.142857,-0.071429,0.100000,0.000000,0.330952,0.242857
8,8,software developer,really nothing share v,everything terrible v,NaN,software developer really nothing share v ever...,18,26,28,0,...,0.000000,0.000000,0.200000,0.200000,1.000000,-1.000000,0.000000,0.000000,0.600000,-0.400000
9,9,everyone high stress company growing pain,world class benefit smart coworkers food galor...,work life balance unrealistic recruiting expec...,stop micromanaging sourcers current recruiting...,everyone high stress company growing pain worl...,56,60,235,238,...,0.540000,0.160000,0.421429,0.257143,0.644444,-0.300000,0.594444,-0.133333,0.566614,-0.069524


In [4]:
negatives = data_prep.negatives.dropna()
negatives

0        job movement lateral extremely difficult get p...
1        hr protect company employee careful restricted...
2        promote work life balance word employee get re...
3        work life balance hostile environment reward b...
4        hectic schedule u ill learning cutting technology
5        hard get often turn away good people left clea...
6                         always enough time meet deadline
7                  limited opportunity move within company
8                                    everything terrible v
9        work life balance unrealistic recruiting expec...
10          growing inertia mono culture better learn love
11                          con find big corporate company
12          getting bureaucratic many reporting goal clear
13                                           find con till
14       getting promoted depends know loud one would n...
15       high pressure bad treatment really bad hour re...
16       sure start bad management bad attitude recogni.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
from nltk import word_tokenize
def tokenize(df):
    tokenized_text = word_tokenize(str(df))
    return tokenized_text

In [10]:
negatives_tokenized = negatives.apply(tokenize)
negatives_tokenized[0]

['job',
 'movement',
 'lateral',
 'extremely',
 'difficult',
 'get',
 'promoted',
 'competing',
 'coworkers',
 'store']

In [11]:
negatives_tokenized

0        [job, movement, lateral, extremely, difficult,...
1        [hr, protect, company, employee, careful, rest...
2        [promote, work, life, balance, word, employee,...
3        [work, life, balance, hostile, environment, re...
4        [hectic, schedule, u, ill, learning, cutting, ...
5        [hard, get, often, turn, away, good, people, l...
6                   [always, enough, time, meet, deadline]
7            [limited, opportunity, move, within, company]
8                                [everything, terrible, v]
9        [work, life, balance, unrealistic, recruiting,...
10       [growing, inertia, mono, culture, better, lear...
11                    [con, find, big, corporate, company]
12       [getting, bureaucratic, many, reporting, goal,...
13                                       [find, con, till]
14       [getting, promoted, depends, know, loud, one, ...
15       [high, pressure, bad, treatment, really, bad, ...
16       [sure, start, bad, management, bad, attitude, .

In [12]:
import gensim
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim

In [15]:
id2word = corpora.Dictionary(negatives_tokenized)
dictionary = id2word
dictionary = dictionary.filter_extremes(no_above=0.85)

texts = negatives_tokenized
corpus = [id2word.doc2bow(text) for text in texts]
ldamallet = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, iterations=100)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()

def compute_coherence_values(dictionary, corpus, texts, limit=8, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=.91)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=8, step=2)

max_y = max(coherence_values)  # Find the maximum y value
max_x = coherence_values.index(max(coherence_values))  # Find the x value corresponding to the maximum y value
#xmax = x[numpy.argmax(y)]

optimal_model = model_list[coherence_values.index(max(coherence_values))]

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word, R=10)
vis

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.077669  0.067305       1        1  20.644650
2     -0.049692  0.126414       2        1  18.237206
4     -0.003911  0.111143       3        1  16.838957
5      0.073260 -0.087587       4        1  15.332829
3      0.122762 -0.116418       5        1  14.785682
1     -0.220088 -0.100857       6        1  14.160684, topic_info=     Category          Freq         Term         Total  loglift  logprob
33    Default  13279.000000         work  13279.000000  10.0000  10.0000
23    Default   4547.000000         hour   4547.000000   9.0000   9.0000
24    Default   3677.000000         life   3677.000000   8.0000   8.0000
21    Default   3000.000000      balance   3000.000000   7.0000   7.0000
29    Default   6499.000000         time   6499.000000   6.0000   6.0000
115   Default   2775.000000         long   2775.000000   5.0000   5.0000
12    Default   6940.000000      company   6940.000000   4.0000   4.0000
223   Default   3246.000000         much   3246.000000   3.0000   3.0000
290   Default   4519.000000         team   4519.000000   2.0000   2.0000
786   Default   1712.000000     politics   1712.000000   1.0000   1.0000
1384   Topic1    123.562889          dog    124.954987   1.5665  -6.8997
5083   Topic1     51.394054   researcher     52.445255   1.5575  -7.7769
4250   Topic1     36.582249     informed     37.442177   1.5545  -8.1169
4772   Topic1     39.343254  measurement     40.395081   1.5513  -8.0441
1657   Topic1     32.041920   assessment     32.915031   1.5508  -8.2494
5150   Topic1     32.121029        nasty     33.010017   1.5504  -8.2469
2753   Topic1     71.442818         wide     73.505646   1.5492  -7.4475
4620   Topic1     29.558216     leveling     30.440525   1.5483  -8.3301
2265   Topic1     27.430502         mate     28.305227   1.5463  -8.4048
825    Topic1     35.778893       taught     37.002510   1.5441  -8.1391
2718   Topic1     54.446568       record     56.524075   1.5403  -7.7192
5030   Topic1     39.968437      trivial     41.380234   1.5430  -8.0283
2387   Topic1    112.289734   evaluation    117.591965   1.5316  -6.9953
987    Topic1    205.409668       rating    218.144135   1.5176  -6.3914
675    Topic1    873.513794  performance    982.558899   1.4601  -4.9439
268    Topic1   1260.089355       review   1475.502319   1.4199  -4.5775
208    Topic1    207.083420         zero    234.039597   1.4553  -6.3833
1397   Topic1     77.603737        score     83.125343   1.5090  -7.3648
51     Topic1    581.232300      support    749.838318   1.3230  -5.3513
290    Topic1   2858.245117         team   4519.735840   1.1195  -3.7585
...       ...           ...          ...           ...      ...      ...
347    Topic5    842.603210         need   2351.196289   0.8853  -4.6461
290    Topic5    951.097839         team   4519.735840   0.3529  -4.5250
4183   Topic6     61.120304   rotational     62.566280   1.9313  -7.2266
3128   Topic6     57.708038      shuttle     59.187431   1.9294  -7.2840
1094   Topic6     34.027042    francisco     34.917622   1.9289  -7.8123
2066   Topic6     25.303787      workday     26.187342   1.9204  -8.1085
5384   Topic6     25.108713  practically     26.054934   1.9177  -8.1162
2417   Topic6     74.529976          bid     77.343849   1.9176  -7.0282
3268   Topic6     39.265762     grueling     40.793358   1.9165  -7.6691
3870   Topic6     27.008385      weekday     28.120634   1.9143  -8.0433
5136   Topic6     23.272057      erratic     24.253595   1.9134  -8.1922
254    Topic6     71.487961         mall     75.229156   1.9037  -7.0699
21     Topic6   2839.098633      balance   3000.509277   1.8994  -3.3882
24     Topic6   3392.736328         life   3677.735352   1.8740  -3.2101
55     Topic6    183.688431       hectic    194.083374   1.8997  -6.1262
23     Topic6   4085.649414         hour   4547.546875   1.8476  -3.0242
1348   Topic6    586.